In [1]:
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.predictor import csv_serializer
import boto3
import re
import os
from sagemaker import get_execution_role

role = get_execution_role()

df=pd.read_csv('s3://sage-maker-4/DadosTeseLogit.csv',sep=',',header=0)
sel=np.where(df.corr()['selected']>.5)[0][0:-1]
df=df.iloc[:,np.concatenate([[30],sel])]

In [9]:
df

,selected,c0_positiva,c0_estimula,c0_recomend
0,1,6.471247,5,6.471247
1,1,6.471247,5,6.471247
2,1,6.471247,5,6.471247
3,1,6.471247,4,6.471247
4,1,6.471247,5,6.471247
5,1,6.471247,5,6.471247
6,1,6.471247,5,6.471247
7,1,6.471247,5,6.471247
8,1,6.471247,5,6.471247
9,1,6.471247,5,6.471247


In [2]:
containers  = {'us-east-1': '123456789.dkr.ecr.us-east-1.amazonaws.com/sklearn'}

In [4]:
bucket =  'sage-maker-4'
prefix = 'sagemaker/xgboost-churn'

In [7]:
df.to_csv('train.csv',header=False, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')

In [12]:
sess =sagemaker.Session()

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=20,
                        min_child_weight=0.1,
                        subsample=0.95,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)



In [ ]:
xgb.fit({'train': s3_input_train})

INFO:sagemaker:Creating training-job with name: sklearn-2018-06-07-03-28-25-935


.................
[I 03:31:03.568 NotebookApp] Writing notebook server cookie secret to /root/.local/share/jupyter/runtime/notebook_cookie_secret
[I 03:31:04.036 NotebookApp] Serving notebooks from local directory: /notebook
[I 03:31:04.036 NotebookApp] 0 active kernels
[I 03:31:04.036 NotebookApp] The Jupyter Notebook is running at:
[I 03:31:04.036 NotebookApp] http://aws:8888/?token=51bc806ff0d4913f317d329040fe31c5b39252b19623ce07
[I 03:31:04.037 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[C 03:31:04.037 NotebookApp] 
    
    Copy/paste this URL into your browser when you connect for the first time,
    to login with a token:
        http://aws:8888/?token=51bc806ff0d4913f317d329040fe31c5b39252b19623ce07&token=51bc806ff0d4913f317d329040fe31c5b39252b19623ce07


In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

In [ ]:
df3=df.iloc[:,[1,2,3]]
predictions =xgb_predictor.predict(df3.as_matrix()).decode('utf-8')
predictions